In [14]:
from glob import glob
import json

import pandas as pd
import numpy as np

## RPC scope reduction

In [15]:
def rpc_scope_reduction(casestudy):
    rpc_scope_res = []
    datapath = '../datasets/{}/workload'.format(casestudy)
    exp = pd.read_csv(datapath + '/experiments.csv', header=None, delimiter=';')
    from_, to = exp.iloc[0, [1, 2]]
    
    with open('../results/{}/workload.json'.format(casestudy)) as f:
        res = json.load(f)
        
    no_pattern = 2
    for frontend in res.keys():
        path = "{}/{}__{}_{}.parquet".format(datapath, frontend, from_, to)
        df = pd.read_parquet(path)
        rpcs =  [c for c in df.columns if c!="traceId" and c!="experiment" and  c!='frontend']
        patset = res[frontend]
        for pat in patset:
            rpcs_ = {pred[0] for pred in pat}
            rpc_scope_res.append(len(rpcs_)/len(rpcs))
            no_pattern += 1
    return rpc_scope_res
        

In [16]:
rpc_scope_res = []
rpc_scope_res += rpc_scope_reduction('trainticket')
rpc_scope_res += rpc_scope_reduction('eshopper')

In [17]:

print("Mean RPC reduction provided by DeLag {:.1%}%".format(np.mean(rpc_scope_res)))

Mean RPC reduction provided by DeLag 10.4%%


## Requests scope reduction

In [18]:
def req_scope_reduction(casestudy):
    datapath = '../datasets/{}/workload'.format(casestudy)
    exp = pd.read_csv(datapath + '/experiments.csv', header=None, delimiter=';')
    from_, to = exp.iloc[0, [1, 2]]
    
    with open('../results/{}/workload.json'.format(casestudy)) as f:
        res = json.load(f)
    req_scope_res = []
    for frontend in res.keys():
        path = "{}/{}__{}_{}.parquet".format(datapath, frontend, from_, to)
        df = pd.read_parquet(path)
        patset = res[frontend]
        for pat in patset:
            df_ = df
            for rpc, e_min, e_max in pat:
                df_ = df_[(e_min <= df_[rpc]) & (df_[rpc] < e_max)]
            req_scope_res.append(len(df_)/len(df))
    return req_scope_res

    

In [19]:
req_scope_res = []
for cs in ['trainticket', 'eshopper']:
    req_scope_res += req_scope_reduction(cs)

In [20]:
print("Mean requests reduction provided by DeLag {:.1%}".format(np.mean(req_scope_res)))

Mean requests reduction provided by DeLag 14.2%
